In [15]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [17]:
data = pd.read_csv('ml-latest-small/ratings.csv')

data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [19]:
# Créer une matrice utilisateur-film
# Chaque ligne représente un utilisateur et chaque colonne représente un film
# Les valeurs sont les notes que les utilisateurs ont attribuées aux films
user_movie_matrix = data.pivot(index='userId', columns='movieId', values='rating').fillna(0)


# Diviser les données en ensembles d'entraînement et de test
# En utilisant 80% des données pour l'entraînement et 20% pour le test
train_data, test_data = train_test_split(user_movie_matrix, test_size=0.2, random_state=42)


# Convertir les ensembles d'entraînement et de test en matrices
train_data_matrix = train_data.values
test_data_matrix = test_data.values

In [21]:
# Définition de l'Autoencodeur
# input_dim est le nombre de films (le nombre de colonnes dans user_movie_matrix)
# encoding_dim est la dimension de la couche cachée, choisie comme 64 pour réduire la dimensionnalité tout en capturant les informations essentielles
input_dim = train_data_matrix.shape[1]
encoding_dim = 64  # Nombre de neurones dans la couche encodée


# Couche d'entrée
input_layer = tf.keras.layers.Input(shape=(input_dim,))


# Couche encodée avec activation ReLU
# ReLU (Rectified Linear Unit) est utilisée ici car elle aide à résoudre le problème de la vanishing gradient et accélère la convergence
encoded = tf.keras.layers.Dense(encoding_dim, activation='relu')(input_layer)


# Couche décodée avec activation Sigmoid
# Sigmoid est utilisée pour ramener les valeurs dans l'intervalle [0, 1], ce qui est approprié car les notes des films sont normalisées
decoded = tf.keras.layers.Dense(input_dim, activation='sigmoid')(encoded)


# Modèle autoencodeur
autoencoder = tf.keras.Model(inputs=input_layer, outputs=decoded)


# Compilation du modèle
# Adam est un optimiseur adaptatif efficace qui fonctionne bien avec les grands datasets et les réseaux de neurones
# La fonction de perte est la Mean Squared Error (MSE) car nous cherchons à minimiser la différence quadratique entre les notes prédites et réelles
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [23]:
# Entraînement de l'Autoencodeur
# epochs: nombre de fois que le modèle va voir l'ensemble des données d'entraînement
# batch_size: nombre de samples que le modèle va voir avant d'ajuster les poids
# shuffle=True permet de mélanger les données d'entraînement pour garantir que le modèle ne mémorise pas l'ordre des données
autoencoder.fit(train_data_matrix, train_data_matrix,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(test_data_matrix, test_data_matrix))

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 283ms/step - loss: 0.4072 - val_loss: 0.4645
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - loss: 0.3937 - val_loss: 0.4439
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - loss: 0.3784 - val_loss: 0.4073
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - loss: 0.3453 - val_loss: 0.3689
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - loss: 0.3039 - val_loss: 0.3377
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - loss: 0.2770 - val_loss: 0.3147
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - loss: 0.2397 - val_loss: 0.2984
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - loss: 0.2213 - val_loss: 0.2871
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - loss: 0.2174 - val_loss: 0.2796
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - loss: 0.1974 - val_loss: 0.2750
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - loss: 0.2011 - val_loss: 0.2714
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - loss: 0.2003 - val_los

In [25]:
# Évaluation du modèle
# La perte (loss) est calculée comme la MSE entre les notes prédites et réelles
loss = autoencoder.evaluate(test_data_matrix, test_data_matrix, verbose=2)
print(f'\nTest Loss: {loss}')


4/4 - 0s - 19ms/step - loss: 0.2560

Test Loss: 0.25597816705703735


In [27]:
# Recommander des films
# Utiliser l'autoencodeur pour obtenir les représentations encodées des films
encoded_movies = autoencoder.predict(test_data_matrix)


# Fonction de recommandation de films
def recommend_movies(user_id, encoded_movies, user_movie_matrix, num_recommendations=5):
    # Trouver l'index de l'utilisateur dans la matrice
    user_index = user_movie_matrix.index.get_loc(user_id)
    
    # Obtenir les notes prédites pour cet utilisateur
    user_ratings = encoded_movies[user_index]
    
    # Trier les films par notes décroissantes
    recommendations = np.argsort(user_ratings)[::-1]
    
    # Trouver les films non vus par l'utilisateur
    unseen_movies = user_movie_matrix.columns[user_movie_matrix.loc[user_id] == 0]
    
    # Recommander les films non vus avec les scores les plus élevés
    recommended_movies = unseen_movies.intersection(recommendations)
    return recommended_movies[:num_recommendations]

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


In [29]:
# Exemple de recommandation pour l'utilisateur 1
recommended_movies = recommend_movies(1, encoded_movies, user_movie_matrix)
print(f"Recommended movies for user 1: {recommended_movies}")

Recommended movies for user 1: Index([2, 4, 5, 7, 8], dtype='int64', name='movieId')


In [31]:
# Lire le fichier films.csv
films_df = pd.read_csv('ml-latest-small/movies.csv')


# Fonction pour récupérer le titre et le genre des films à partir d'une liste d'IDs de films
def get_film_details(film_ids, films_df):
    details = films_df[films_df['movieId'].isin(film_ids)][['title', 'genres']]
    return details

In [33]:
# Exemple d'utilisation avec un tableau d'IDs de films
film_details = get_film_details(recommended_movies, films_df)

print(film_details)

                                title                      genres
1                      Jumanji (1995)  Adventure|Children|Fantasy
3            Waiting to Exhale (1995)        Comedy|Drama|Romance
4  Father of the Bride Part II (1995)                      Comedy
6                      Sabrina (1995)              Comedy|Romance
7                 Tom and Huck (1995)          Adventure|Children
